In [1]:
# =============================================================================
# ЯЧЕЙКА 1: НАСТРОЙКА И ЗАГРУЗКА ДАННЫХ
# =============================================================================

# --- 1. Базовые импорты ---
import logging
import sys
import os
import pandas as pd
from datetime import datetime

# --- 2. Импорт для загрузки .env файла ---
from dotenv import load_dotenv

# --- 3. Импорт наших модулей ---
from htx_api import HtxApi
from trade_logger import TradeLogger
from arbitrage_strategy import TriangularArbitrageStrategy
import config

# --- 4. Настройка логирования ---
log_formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler = logging.FileHandler('backtest.log', mode='w')
file_handler.setFormatter(log_formatter)
stream_handler = logging.StreamHandler(sys.stdout)
stream_handler.setFormatter(log_formatter)
stream_handler.setLevel(logging.WARNING) 
root_logger = logging.getLogger()
root_logger.setLevel(logging.INFO)
root_logger.handlers.clear()
root_logger.addHandler(file_handler)
root_logger.addHandler(stream_handler)
logging.getLogger('huobi').setLevel(logging.WARNING)
print("Логирование настроено. Подробный вывод в файле backtest.log")

# --- 5. Загрузка переменных из .env файла ---
load_dotenv()
print("Попытка загрузки переменных из .env файла...")

# --- 6. Загрузка ключей и инициализация API ---
API_KEY = os.getenv('API_KEY')
SECRET_KEY = os.getenv('SECRET_KEY')
BASE_URL = os.getenv('BASE_URL')

if not all([API_KEY, SECRET_KEY, BASE_URL]):
    raise ValueError("ОШИБКА: Не удалось загрузить API_KEY, SECRET_KEY или BASE_URL. Проверьте ваш .env файл.")
else:
    print("Ключи API успешно загружены.")

htx_api = HtxApi(api_key=API_KEY, secret_key=SECRET_KEY, base_url=BASE_URL)
print("Клиент HTX API создан.")

# --- 7. Функция для загрузки и кэширования данных ---
def download_and_cache_data(api, symbols, period, size, date_str):
    """
    Загружает данные для всех символов за указанную дату из кэша.
    Если данные для хотя бы одного символа отсутствуют, скачивает
    СВЕЖИЕ данные для ВСЕХ символов и сохраняет их с ТЕКУЩЕЙ датой.
    """
    data_dir = 'historical_data'
    os.makedirs(data_dir, exist_ok=True)
    
    historical_data = {}
    all_files_exist_for_date = True
    
    # Сначала проверяем, существуют ли файлы для ВСЕХ символов за нужную дату
    for symbol in symbols:
        file_path = os.path.join(data_dir, f"{symbol}_{date_str}.csv")
        if not os.path.exists(file_path):
            all_files_exist_for_date = False
            break
            
    # Если все файлы на месте, загружаем их из кэша
    if all_files_exist_for_date:
        print(f"Загрузка данных из кэша за {date_str}...")
        for symbol in symbols:
            file_path = os.path.join(data_dir, f"{symbol}_{date_str}.csv")
            df = pd.read_csv(file_path, index_col='timestamp', parse_dates=True)
            df.index = pd.to_datetime(df.index, unit='s')
            historical_data[symbol] = df
        return historical_data
        
    # Если хотя бы одного файла нет, скачиваем свежие данные для всех
    else:
        print(f"Данные за {date_str} не найдены в кэше. Загрузка свежих данных с биржи...")
        
        today_str = datetime.now().strftime('%Y-%m-%d')
        print(f"Новые данные будут сохранены с сегодняшней датой: {today_str}")
        
        for symbol in symbols:
            print(f"  - Загрузка {symbol}...")
            klines = api.get_historical_kline(symbol, period, size)
            
            if klines:
                file_path = os.path.join(data_dir, f"{symbol}_{today_str}.csv")
                df = pd.DataFrame(klines)
                df.set_index('timestamp', inplace=True)
                df.to_csv(file_path)
                
                df_loaded = pd.read_csv(file_path, index_col='timestamp', parse_dates=True)
                df_loaded.index = pd.to_datetime(df_loaded.index, unit='s')
                historical_data[symbol] = df_loaded
            else:
                print(f"Не удалось загрузить данные для {symbol}. Операция прервана.")
                return None
                
        return historical_data





Логирование настроено. Подробный вывод в файле backtest.log
Попытка загрузки переменных из .env файла...
Ключи API успешно загружены.
Клиент HTX API создан.


In [2]:
# =============================================================================
# ЯЧЕЙКА 2: ЗАПУСК БЭКТЕСТА (ИНТЕРАКТИВНАЯ ВЕРСИЯ)
# =============================================================================
from datetime import datetime

# --- 1. Запрашиваем дату у пользователя ---
# Предлагаем сегодняшнюю дату как значение по умолчанию
today_str = datetime.now().strftime('%Y-%m-%d')
prompt = f"Введите дату для теста в формате ГГГГ-ММ-ДД (нажмите Enter, чтобы использовать сегодняшнюю: {today_str}): "
test_date_str = input(prompt)

# Если пользователь ничего не ввел, используем дату по умолчанию
if not test_date_str:
    test_date_str = today_str

print(f"Выбрана дата для теста: {test_date_str}")

# --- 2. Определяем символы ---
symbols = ['btcusdt', 'ethusdt', 'ethbtc'] 

# --- 3. Загружаем данные (из кэша или с биржи) ---
# Загружаем 2000 минутных свечей
historical_data = download_and_cache_data(htx_api, symbols, '1min', 2000, test_date_str)

# --- 4. Запуск бэктеста (если данные успешно загружены) ---
if historical_data:
    print("\nЗапуск бэктеста...")
    
    # Создаем логгер без аргументов.
    # Он сам создаст нужный файл в папке /res.
    trade_logger = TradeLogger()

    strategy = TriangularArbitrageStrategy(
        data=historical_data,
        logger=trade_logger,
        min_profit_threshold=config.MIN_PROFIT_THRESHOLD,
        position_size=config.POSITION_SIZE,
        fee_rate=0.002 
    )

    # Запускаем основной цикл стратегии
    strategy.start()
    strategy.run()

    print("Бэктест завершен.")
else:
    print("Бэктест не может быть запущен, так как данные не были загружены.")

Выбрана дата для теста: 2025-07-28
Данные за 2025-07-28 не найдены в кэше. Загрузка свежих данных с биржи...
Новые данные будут сохранены с сегодняшней датой: 2025-08-02
  - Загрузка btcusdt...
call_sync  === recv data :  {'ch': 'market.btcusdt.kline.1min', 'status': 'ok', 'ts': 1754123807805, 'data': [{'id': 1754123760, 'open': 113765.86, 'close': 113798.57, 'low': 113765.86, 'high': 113798.57, 'amount': 0.5892796590725606, 'vol': 67048.09636556, 'count': 380}, {'id': 1754123700, 'open': 113753.86, 'close': 113765.01, 'low': 113750.01, 'high': 113765.01, 'amount': 0.3071618946062918, 'vol': 34940.17740967, 'count': 136}, {'id': 1754123640, 'open': 113794.0, 'close': 113752.86, 'low': 113745.85, 'high': 113882.01, 'amount': 14.227030004297136, 'vol': 1619252.4042719044, 'count': 9735}, {'id': 1754123580, 'open': 113816.01, 'close': 113794.85, 'low': 113794.85, 'high': 113817.01, 'amount': 0.2641564343730759, 'vol': 30061.703021235528, 'count': 1088}, {'id': 1754123520, 'open': 113804.0

/tmp/ipykernel_20309/3410024438.py:100: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_loaded = pd.read_csv(file_path, index_col='timestamp', parse_dates=True)


call_sync  === recv data :  {'ch': 'market.ethusdt.kline.1min', 'status': 'ok', 'ts': 1754123812094, 'data': [{'id': 1754123760, 'open': 3498.17, 'close': 3499.07, 'low': 3497.44, 'high': 3502.3, 'amount': 69.63090591460613, 'vol': 243639.295595, 'count': 63}, {'id': 1754123700, 'open': 3494.99, 'close': 3498.02, 'low': 3494.99, 'high': 3498.76, 'amount': 207.3614, 'vol': 724805.165519, 'count': 229}, {'id': 1754123640, 'open': 3493.38, 'close': 3495.0, 'low': 3486.63, 'high': 3496.14, 'amount': 90.88060057307796, 'vol': 317397.209737, 'count': 93}, {'id': 1754123580, 'open': 3499.4, 'close': 3493.38, 'low': 3493.38, 'high': 3499.77, 'amount': 24.6268, 'vol': 86124.151194, 'count': 69}, {'id': 1754123520, 'open': 3500.59, 'close': 3499.58, 'low': 3499.58, 'high': 3501.49, 'amount': 118.988, 'vol': 416527.84647, 'count': 29}, {'id': 1754123460, 'open': 3505.62, 'close': 3499.81, 'low': 3498.59, 'high': 3505.62, 'amount': 20.8313, 'vol': 72955.109045, 'count': 74}, {'id': 1754123400, 'op

/tmp/ipykernel_20309/3410024438.py:100: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_loaded = pd.read_csv(file_path, index_col='timestamp', parse_dates=True)


call_sync  === recv data :  {'ch': 'market.ethbtc.kline.1min', 'status': 'ok', 'ts': 1754123814269, 'data': [{'id': 1754123760, 'open': 0.030734, 'close': 0.030734, 'low': 0.030734, 'high': 0.030734, 'amount': 0.0049, 'vol': 0.0001505966, 'count': 1}, {'id': 1754123700, 'open': 0.030739, 'close': 0.030739, 'low': 0.030739, 'high': 0.030739, 'amount': 0.01, 'vol': 0.00030739, 'count': 2}, {'id': 1754123640, 'open': 0.030715, 'close': 0.030717, 'low': 0.030673, 'high': 0.030717, 'amount': 0.5425, 'vol': 0.0166605642, 'count': 7}, {'id': 1754123580, 'open': 0.030784, 'close': 0.030784, 'low': 0.030784, 'high': 0.030784, 'amount': 0.0, 'vol': 0.0, 'count': 0}, {'id': 1754123520, 'open': 0.030784, 'close': 0.030784, 'low': 0.030784, 'high': 0.030784, 'amount': 0.0, 'vol': 0.0, 'count': 0}, {'id': 1754123460, 'open': 0.030784, 'close': 0.030784, 'low': 0.030784, 'high': 0.030784, 'amount': 0.0105, 'vol': 0.000323232, 'count': 1}, {'id': 1754123400, 'open': 0.030826, 'close': 0.030826, 'low':

/tmp/ipykernel_20309/3410024438.py:100: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_loaded = pd.read_csv(file_path, index_col='timestamp', parse_dates=True)


Сессия началась: 2025-08-02 10:36:55
Начальный баланс: 15.00 USDT
--------------------------------------------------------------------------------

--------------------------------------------------------------------------------
Сессия завершена: 2025-08-02 10:36:55
Продолжительность: 0:00:00
Итоговый баланс: 15.0000 USDT
Общая чистая прибыль: +0.0000 USDT

Бэктест завершен.
